In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.models import resnet18
import tqdm 
from sklearn.metrics import f1_score, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 이미지 전처리 및 데이터로더 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['normal', 'label1', 'label2','label3','label4','label5','label6','label7','label8']
        self.data = datasets.ImageFolder(root=root_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        return img, label

# 데이터 전처리 및 변환
transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset(root_dir='./melspec/train', transform=transform)
test_dataset = CustomDataset(root_dir='./melspec/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0, drop_last=True)

# ResNet-18 모델 생성
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# 모델 및 손실 함수, 옵티마이저를 GPU로 이동
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, test_loader, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in tqdm.tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # 매 에포크 끝에 테스트 데이터셋을 사용하여 F1 score 계산 및 출력
        test_f1_score = evaluate_f1_score(model, test_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], mfcc+ pca Test F1 Score: {test_f1_score:.4f}')

# 테스트 데이터셋을 사용하여 F1 score를 계산하는 함수 정의
def evaluate_f1_score(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in tqdm.tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    f1 = f1_score(all_labels, all_predictions, average='weighted')  # 수정 가능한 average 매개변수
    return f1

# 학습 및 테스트 수행
train(model, train_loader, criterion, optimizer, test_loader, num_epochs=100)


Using device: cuda


c:\Users\user\anaconda3\envs\mintorch1221\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\mintorch1221\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1125/1125 [02:59<00:00,  6.27it/s]


Epoch [1/100], mfcc+ pca Test F1 Score: 0.9292


100%|██████████| 1125/1125 [01:42<00:00, 11.03it/s]


Epoch [2/100], mfcc+ pca Test F1 Score: 0.9582


...

Epoch [10/100], mfcc+ pca Test F1 Score: 0.9776


100%|██████████| 1125/1125 [02:08<00:00,  8.74it/s]


Epoch [11/100], mfcc+ pca Test F1 Score: 0.9878


  6%|▌         | 682/11250 [02:02<31:33,  5.58it/s]  


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 22915200 bytes.